In [1]:
import cv2     # for capturing videos
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images

Using TensorFlow backend.
/Users/phunp/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/phunp/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/phunp/opt/anaconda3/envs/py37/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/phunp/o

In [2]:
count = 0
videoFile = "../real_data/2010-09-13T13-15-43.wmv"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
# frameRate = cap.get(5) #frame rate
# x=1
# while(cap.isOpened()):
#     frameId = cap.get(1) #current frame number
#     ret, frame = cap.read()
#     if (ret != True):
#         break
#     if (frameId % math.floor(frameRate) == 0):
#         filename ="frame%d.jpg" % count;count+=1
#         cv2.imwrite(filename, frame)
# cap.release()
# print ("Done!")
success, image = cap.read()
count = 0
while success:
  cv2.imwrite("all-frames/frame__%d.jpg" % count, image)     # save frame as JPEG file      
  success, image = cap.read()
#   print('Read a new frame: ', success)
  count += 1
cap.release()
print("Capture total of " + str(count))

Capture total of 872


In [10]:
import os

PERCENT_SKIP_HEAD = 18
PERCENT_SKIP_TAIL = 12
CAPTURING_INTERVAL = 10
DEFAULT_FPS = 30


def get_frame_rate(video):
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
    if int(major_ver)  < 3:
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))
    return fps

def extract_frames(pathIn, pathOut):
    count = 0
    vidcap = cv2.VideoCapture(pathIn)
#     print(vidcap.isOpened())
#     getFrameRate(vidcap)
#     success,image = vidcap.read()
    success = True
#     print("frame count %d" % int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)))
    videoDuration = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT)) / int(vidcap.get(cv2.CAP_PROP_FPS))
#     print("video duration is %d" % videoDuration)
#     baseName = os.path.basename(pathIn)
    skipHead = int((PERCENT_SKIP_HEAD * videoDuration * DEFAULT_FPS)/100)
    skipTail = int(((100 - PERCENT_SKIP_TAIL) * videoDuration * DEFAULT_FPS)/100)
    capturingInterval = int((CAPTURING_INTERVAL * videoDuration * DEFAULT_FPS)/100)
#     print("skipHead %d, tail %d, interval %d" % (skipHead, skipTail, capturingInterval))
    imgNum = 0
    while success:
#         vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line 
        success,image = vidcap.read()
        if count > skipHead and count < skipTail and count >= (skipHead + (capturingInterval * imgNum)):
            cv2.imwrite( pathOut + "__%d.jpg" % imgNum, image)     # save frame as JPEG file
            imgNum += 1
        count = count + 1
    vidcap.release()
#     print("end at count " + str(count))

extract_frames("../real_data/2010-09-13T13-15-43.wmv", "1sec-frame/XXX_COMPUTED_YYY")
print("Done")

Done


In [11]:
path = "/Users/phunp/Documents/workspace/thesis_data/CD_DATA/"

import glob

count = 0
distinctFileNames = set()
for filename in glob.iglob(path + '**/*.wmv', recursive=True):
#     print(os.path.basename(filename))
    baseName = os.path.basename(filename)
    if baseName in distinctFileNames:
        print("duplicated file name " + filename)
    else:
        distinctFileNames.add(os.path.basename(filename))
    count += 1

print("there are total of %d files, distinct count %d" % (count, len(distinctFileNames)))

there are total of 4454 files, distinct count 4454


In [12]:
import pandas as pd
import re

PATTERN_VID = '^[0-9]{2}[.][0-9]{2}[.][0-9]{2}[.][0-9]{2}[.][0-9]{2}[.][0-9]{2}$'
PATTERN_RES = '^[1-7]$'

def is_valid_vid_string(vid_str):
    return vid_str is not None and bool(re.match(PATTERN_VID, vid_str))


def is_valid_result_string(res_str):
    return res_str is not None and bool(re.match(PATTERN_RES, res_str))


LABEL_EDID = "EDID"
LABEL_L_VO = "L_VO"
LABEL_R_VO = "R_VO"
LABEL_L_Middle_E = "L_Middle_Res"
LABEL_R_Middle_E = "R_Middle_Res"
LABEL_VISIT_NO = "Visit_no"

df = pd.read_excel(r'/Users/phunp/Documents/Flinders/Thesis/data/real_data/SwimmingPoolData_from Charmaine/CLEAN_ALL.xlsx', 
                   header=4, converters={
                        LABEL_EDID: str,
                        LABEL_L_VO: str,
                        LABEL_R_VO: str,
                        LABEL_L_Middle_E: str,
                        LABEL_R_Middle_E: str,
                        LABEL_VISIT_NO: str})
# print(df)
# print(df.dtypes)
# df.info()

class RawData:
    def __init__(self, edid, visit_no, l_vid, r_vid, l_me_res, r_me_res):
        self.edid = edid
        self.visit_no = visit_no
        self.l_vid = l_vid
        self.r_vid = r_vid
        self.l_me_res = l_me_res
        self.r_me_res = r_me_res
        self.r_vid_loc = None
        self.l_vid_loc = None

    def is_valid(self):
        return self.edid is not None and len(self.edid) > 0 \
            and self.visit_no is not None and len(self.visit_no) > 0 \
            and is_valid_vid_string(self.l_vid) \
            and is_valid_vid_string(self.r_vid) \
            and is_valid_result_string(self.l_me_res) \
            and is_valid_result_string(self.r_me_res)
    def has_vid_locs(self):
        return self.r_vid_loc is not None and len(self.r_vid_loc) > 0\
            and self.l_vid_loc is not None and len(self.l_vid_loc) > 0
    

totalRecords = 0
invalidRecords = 0
validRecords = 0

DataRaws = {}

for index, row in df.iterrows():
    # print(row[LABEL_EDID], row[LABEL_L_VO], row[LABEL_R_VO], row[LABEL_L_Middle_E], row[LABEL_R_Middle_E])
    totalRecords += 1
    rawData = RawData(str(row[LABEL_EDID]), str(row[LABEL_VISIT_NO]), str(row[LABEL_L_VO]), str(row[LABEL_R_VO]), str(row[LABEL_L_Middle_E]), str(row[LABEL_R_Middle_E]))
    if rawData.is_valid():
        validRecords += 1
        DataRaws[rawData.edid + "-" + rawData.visit_no] = rawData
    else:
        invalidRecords += 1

print("total record %d, valid record %d, invalid record %d" % (totalRecords, validRecords, invalidRecords))
print("size of total record, distinct by EDID: ", len(DataRaws))

total record 2088, valid record 1776, invalid record 312
size of total record, distinct by EDID:  1776


In [15]:
import os
import glob

allRawFiles = {}
files = glob.glob(r'/Users/phunp/Documents/workspace/thesis_data/CD_DATA/**/*.wmv', recursive=True)
for file in files:
    baseNameNoExt = os.path.splitext(os.path.basename(file))[0]
    if baseNameNoExt not in allRawFiles:
        allRawFiles[baseNameNoExt] = [file]
    else:
        allRawFiles[baseNameNoExt].append(file)
        
print("total of " + str(len(allRawFiles)))

for baseName, listFiles in allRawFiles.items():
    if len(listFiles) > 1:
        # rename with dup2
        # /Users/phunp/Documents/workspace/thesis_data/CD_DATA/CD9/VO1_TRIP_03_2010/VO1_ernabella_02_1002/2010-09-09T13-13-28.wmv
        # /Users/phunp/Documents/workspace/thesis_data/CD_DATA/CD6/VO1_TRIP_01_2010/VO1_mimili_01_1001/2010-03-22T13-01-28.wmv
        # /Users/phunp/Documents/workspace/thesis_data/CD_DATA/CD1/V01_TRIP_01/VO1_Murputja/2009-03-24T13-18-08.wmv
        print("Files name " + baseName + " has " + str(len(listFiles)) + " files in sources")
        for fl in listFiles:
            print(fl)

def convert_short_to_long_vid_name(short_vid_name):
    parts = str.split(short_vid_name, '.')
    if len(parts) != 6:
        print("Not a valid short vid name: ", short_vid_name)
        return None
    else:
        return "20" + parts[0] + "-" + parts[1] + "-" + parts[2] + "T" + parts[3] + "-" + parts[4] + "-" + parts[5]


print(convert_short_to_long_vid_name("123123123"))
print(convert_short_to_long_vid_name("10.03.16.13.28.09"))


total of 4454
Not a valid short vid name:  123123123
None
2010-03-16T13-28-09


In [16]:
leftMiss = 0
rightMiss = 0
allMiss = 0
for key, dataRaw in DataRaws.items():
    if not rawData.is_valid():
        continue
    lVidBaseName = convert_short_to_long_vid_name(dataRaw.l_vid)
    rVidBaseName = convert_short_to_long_vid_name(dataRaw.r_vid)
    rm = lm = False
    if lVidBaseName not in allRawFiles:
#         print("LEFT File not found for %s: %s" % (dataRaw.edid, lVidBaseName))
        leftMiss += 1
        lm = True
    else:
        dataRaw.l_vid_loc = allRawFiles[lVidBaseName][0]
    if rVidBaseName not in allRawFiles:
#         print("RIGHT File not found for %s: %s" % (dataRaw.edid, rVidBaseName))
        rightMiss += 1
        rm = True
    else:
        dataRaw.r_vid_loc = allRawFiles[rVidBaseName][0]
    if rm and lm:
        allMiss += 1

print("All miss %d, left %d, right %d" % (allMiss, leftMiss, rightMiss))

All miss 308, left 354, right 366


In [27]:
def print_left_done_data(rawData):
    print("-- %s,%s,L,%s,%s" % (rawData.edid, rawData.visit_no, rawData.l_vid, rawData.l_me_res))

def print_right_done_data(rawData):
    print("-- %s,%s,R,%s,%s" % (rawData.edid, rawData.visit_no, rawData.r_vid, rawData.r_me_res))
    
count = 0
for key, rawData in DataRaws.items():
    if not rawData.is_valid():
        print("Continue")
        continue
    count += 1
    if count % 50 == 0:
        print("........ DONE FOR " + str(count) + " .....")
    if rawData.l_vid_loc is not None and len(rawData.l_vid_loc) > 0:
        baseNameNoExt = os.path.splitext(os.path.basename(rawData.l_vid_loc))[0]
        extract_frames(rawData.l_vid_loc, "/Users/phunp/Documents/Flinders/Thesis/data/real_data/extract_data/" + key + "-L--" + rawData.l_me_res + "--" + baseNameNoExt)
        print_left_done_data(rawData)
        
    if rawData.r_vid_loc is not None and len(rawData.r_vid_loc) > 0:
        baseNameNoExt = os.path.splitext(os.path.basename(rawData.r_vid_loc))[0]
        extract_frames(rawData.r_vid_loc, "/Users/phunp/Documents/Flinders/Thesis/data/real_data/extract_data/" + key + "-R--" + rawData.l_me_res + "--" + baseNameNoExt)
        print_right_done_data(rawData)
        
print("Done!!!! for " + str(count))

-- 101748836L,6,L,11.09.13.12.44.11,7
-- 101748836L,6,R,11.09.13.12.44.59,7
-- 103666734P,4,L,10.09.07.12.04.42,3
-- 103666734P,4,R,10.09.07.12.05.31,3
-- 103666734P,1,L,09.03.24.14.25.04,5
-- 103666734P,1,R,09.03.24.14.27.20,5
-- 102991594P,4,L,10.09.07.12.01.52,1
-- 102991594P,4,R,10.09.07.12.02.51,1
-- 102911341N,2,L,09.09.15.10.06.49,3
-- 102911341N,2,R,09.09.15.10.04.48,2
-- 100512912X,2,L,09.09.15.10.14.32,2
-- 100512912X,2,R,09.09.15.10.15.24,3
-- 100799943K,6,L,11.09.13.10.22.36,2
-- 100799943K,6,R,11.09.13.10.21.22,4
-- 102155371N,6,L,11.09.13.10.11.01,2
-- 102155371N,6,R,11.09.13.10.12.37,2
-- 102963534P,3,L,10.03.17.12.00.18,5
-- 102963534P,3,R,10.03.17.12.02.03,5
-- 101748836L,1,L,09.03.24.14.15.32,4
-- 101748836L,1,R,09.03.24.14.14.20,4
-- 100156440J,2,L,09.09.15.09.58.03,1
-- 100156440J,2,R,09.09.15.09.59.25,1
-- 101729127F,4,L,10.09.07.11.20.43,2
-- 101729127F,4,R,10.09.07.11.21.25,2
-- 102155371N,2,L,09.09.15.10.24.49,1
-- 102155371N,2,R,09.09.15.10.24.00,1
-- 101042563

-- 104423052X,6,R,11.09.16.11.27.57,2
-- 103037641N,1,L,09.03.27.13.43.36,2
-- 103037641N,1,R,09.03.27.13.42.11,5
-- 104617290A,5,L,11.03.25.12.10.49,2
-- 104617290A,5,R,11.03.25.12.11.13,2
-- 100783620J,3,L,10.03.15.14.45.25,2
-- 100783620J,3,R,10.03.15.14.45.53,2
-- 102715305A,4,L,10.09.08.10.50.13,1
-- 101170769M,4,L,10.09.09.12.47.00,3
-- 101170769M,4,R,10.09.09.12.46.05,2
-- 100783620J,1,L,09.03.27.14.36.26,1
-- 100783620J,1,R,09.03.27.14.35.09,2
-- 103378890J,2,L,09.09.18.11.46.54,3
-- 103378890J,2,R,09.09.18.11.47.42,3
-- 100336309M,2,L,09.09.18.10.57.48,2
-- 100336309M,2,R,09.09.18.10.57.00,1
-- 104375209M,4,L,10.09.08.13.27.55,1
-- 104375209M,4,R,10.09.08.13.27.21,1
-- 103604443K,1,L,09.03.30.10.35.43,2
-- 103604443K,1,R,09.03.30.10.35.11,1
-- 101909950J,4,L,10.09.09.09.49.14,2
-- 101909950J,4,R,10.09.09.09.50.24,4
-- 102212277F,4,L,10.09.09.10.09.50,2
-- 102212277F,4,R,10.09.09.10.09.13,2
-- 101559670A,3,L,10.03.15.16.18.23,5
-- 101559670A,3,R,10.03.15.16.18.51,2
-- 101610988

-- 103506370J,4,L,10.09.08.12.21.20,2
-- 103506370J,4,R,10.09.08.12.22.35,1
-- 102519969M,4,L,10.09.09.13.47.24,3
-- 102519969M,4,R,10.09.09.13.48.07,2
-- 101523475A,1,L,09.03.30.14.51.50,2
-- 101523475A,1,R,09.03.30.14.53.50,2
-- 101887258T,1,L,09.03.30.10.28.01,5
-- 101887258T,1,R,09.03.30.10.27.34,5
-- 102972752X,4,L,10.09.08.10.09.02,5
-- 102972752X,4,R,10.09.08.10.10.32,1
-- 103141721N,2,L,09.09.18.13.06.28,2
-- 103141721N,2,R,09.09.18.13.05.24,1
-- 103141721N,6,L,11.09.16.10.36.58,4
-- 103141721N,6,R,11.09.16.10.38.31,4
-- 103227276L,2,L,09.09.21.11.24.34,5
-- 103227276L,2,R,09.09.21.11.26.04,5
-- 100696038T,3,L,10.03.16.10.55.25,3
-- 100696038T,3,R,10.03.16.10.55.53,5
-- 101648604P,4,L,10.09.09.14.53.43,2
-- 101648604P,4,R,10.09.09.14.52.45,1
-- 100561654P,4,L,10.09.09.12.09.21,1
-- 100561654P,4,R,10.09.09.12.08.18,1
-- 103227290A,4,L,10.09.08.12.10.34,2
-- 103227290A,4,R,10.09.08.12.11.28,2
-- 104481504P,6,L,11.09.19.10.20.27,4
-- 104481504P,6,R,11.09.19.10.21.17,4
-- 102528849

-- 102968947F,4,R,10.09.08.11.05.34,1
-- 100588629M,5,L,11.03.25.10.57.01,2
-- 100588629M,5,R,11.03.25.10.57.16,2
-- 100168045A,5,L,11.03.25.14.41.46,2
-- 100168045A,5,R,11.03.25.14.41.59,2
-- 103310441N,3,L,10.03.16.12.16.02,3
-- 103310441N,3,R,10.03.16.12.16.31,5
-- 100118183K,1,L,09.03.30.10.05.45,2
-- 100118183K,1,R,09.03.30.10.03.25,2
-- 104118360A,6,L,11.09.16.12.21.52,3
-- 104118360A,6,R,11.09.16.12.23.01,2
-- 101448907F,3,L,10.03.15.14.34.35,7
-- 102968947F,6,L,11.09.19.10.05.22,2
-- 102968947F,6,R,11.09.19.10.06.31,2
-- 103378890J,3,L,10.03.15.16.24.15,3
-- 103378890J,3,R,10.03.15.16.25.00,5
-- 102268814P,1,R,09.03.30.12.45.03,3
-- 103310441N,1,L,09.03.30.09.35.53,3
-- 103310441N,1,R,09.03.30.09.34.59,3
-- 100865686L,2,L,09.09.21.10.34.33,3
-- 100865686L,2,R,09.09.21.10.33.02,3
-- 102345302X,4,L,10.09.09.13.53.43,2
-- 102345302X,4,R,10.09.09.13.52.54,1
-- 101994170A,4,L,10.09.09.10.16.39,2
-- 101994170A,4,R,10.09.09.10.15.32,2
-- 101853279M,4,L,10.09.09.12.12.52,5
-- 102528849

-- 102516771N,3,R,10.03.15.11.18.18,7
-- 102516734P,3,L,10.03.15.11.29.19,4
-- 102516734P,3,R,10.03.15.11.29.50,4
-- 101137620J,3,L,10.03.15.12.01.25,7
-- 101137620J,3,R,10.03.15.12.01.44,2
-- 102345302X,6,L,11.09.16.10.54.21,4
-- 102345302X,6,R,11.09.16.10.55.08,2
-- 104481669M,6,L,11.09.16.12.14.51,2
-- 104481669M,6,R,11.09.16.12.16.04,7
-- 101728837F,2,L,09.09.18.10.26.24,2
-- 101728837F,2,R,09.09.18.10.27.03,1
-- 101137620J,2,L,09.09.18.10.44.08,1
-- 101137620J,2,R,09.09.18.10.45.25,1
-- 103018699M,1,L,09.03.27.13.19.23,2
-- 103018699M,1,R,09.03.27.13.18.10,3
-- 101637130A,1,L,09.03.27.11.52.02,4
-- 101637130A,1,R,09.03.27.11.53.38,4
-- 100251037F,1,L,09.03.30.15.48.17,4
-- 100251037F,1,R,09.03.30.15.49.10,4
-- 103296048T,1,L,09.03.27.11.39.25,4
-- 103296048T,1,R,09.03.27.11.37.20,4
-- 101866773K,5,R,11.03.25.14.40.06,4
-- 101866773K,2,L,09.09.18.12.28.18,2
-- 103227319M,1,L,09.03.30.10.19.49,4
-- 101036808T,1,L,09.03.30.13.08.50,5
-- 101036808T,1,R,09.03.30.13.10.12,5
-- 100897985

...

-- 102207010J,2,L,09.09.14.09.57.53,1
-- 102207010J,2,R,09.09.14.09.57.01,1
-- 102204517F,1,L,09.03.23.14.12.13,1
-- 102204517F,1,R,09.03.23.14.10.39,1
-- 103672568T,3,L,10.03.19.11.39.39,7
-- 103672568T,3,R,10.03.19.11.40.03,7
-- 100763381N,1,L,09.03.23.13.10.51,5
-- 100763381N,1,R,09.03.23.13.07.28,5
-- 103980400J,2,L,09.09.14.10.41.54,1
-- 103980400J,2,R,09.09.14.10.43.28,1
-- 103137768T,1,R,09.03.23.11.59.09,2
-- 104424050A,3,L,10.03.19.10.48.03,2
-- 104424050A,3,R,10.03.19.10.49.02,2
-- 101270800J,4,R,10.09.06.13.16.11,1
-- 104058136L,3,L,10.03.19.11.44.46,3
-- 104642967F,5,L,11.03.21.12.41.39,2
-- 104642967F,5,R,11.03.21.12.41.15,2
-- 102706799M,1,L,09.03.23.11.11.26,5
-- 102706799M,1,R,09.03.23.11.14.09,5
-- 102967938T,1,L,09.03.23.12.15.39,1
-- 102967938T,1,R,09.03.23.12.13.53,1
-- 100150091N,3,L,10.03.19.10.58.44,7
-- 100150091N,3,R,10.03.19.10.59.26,7
-- 102046246L,1,L,09.03.23.12.39.20,2
-- 102046246L,1,R,09.03.23.12.40.52,2
-- 102046246L,2,L,09.09.14.14.07.48,1
-- 102046246

-- 103655387F,6,L,11.09.14.11.08.16,2
-- 103655387F,6,R,11.09.14.11.07.28,2
-- 103005970A,6,L,11.09.14.11.22.01,5
-- 103005970A,6,R,11.09.14.11.20.56,5
-- 103388084P,3,L,10.03.17.12.08.32,1
-- 103388084P,3,R,10.03.17.12.08.50,1
-- 102723700A,3,L,10.03.17.14.27.10,2
-- 102723700A,3,R,10.03.17.14.27.37,2
-- 102206707F,1,L,09.03.25.10.51.01,7
-- 102206707F,1,R,09.03.25.10.53.41,2
-- 103388084P,1,L,09.03.25.11.15.49,1
-- 103388084P,1,R,09.03.25.11.17.19,1
-- 103335724P,3,L,10.03.17.12.53.08,5
-- 103335724P,3,R,10.03.17.12.53.35,3
-- 103514173K,3,L,10.03.17.13.05.20,1
-- 103514173K,3,R,10.03.17.13.05.50,1
-- 100363244P,6,L,11.09.14.09.36.07,7
-- 100363244P,6,R,11.09.14.09.35.26,7
-- 101474581N,2,L,09.09.16.12.31.50,5
-- 101474581N,2,R,09.09.16.12.29.59,5
-- 102969772X,3,L,10.03.17.13.06.43,7
-- 102969772X,3,R,10.03.17.13.07.00,2
-- 102591408T,1,L,09.03.25.11.07.51,5
-- 102591408T,1,R,09.03.25.11.10.15,5
-- 102591408T,2,L,09.09.16.10.54.01,5
-- 102591408T,2,R,09.09.16.10.53.14,5
........ DON

-- 104041271N,6,L,11.09.20.09.38.40,2
-- 104041271N,6,R,11.09.20.09.37.21,2
-- 101812236L,2,L,09.09.22.09.57.21,1
-- 101812236L,2,R,09.09.22.09.58.12,1
-- 103646286L,3,L,10.03.22.12.44.10,5
-- 103646286L,3,R,10.03.22.12.44.46,5
-- 103868749M,2,R,09.09.21.13.14.52,2
-- 101333735A,1,L,09.03.31.13.02.03,2
-- 103868749M,1,L,09.03.31.14.38.17,3
-- 103868749M,1,R,09.03.31.14.37.53,2
-- 102693161N,5,L,11.03.29.12.32.33,2
-- 102693161N,5,R,11.03.29.12.33.56,5
-- 100694807F,3,L,10.03.22.11.26.13,2
-- 100694807F,3,R,10.03.22.11.26.46,2
-- 101952153K,5,L,11.03.29.10.28.00,3
-- 101952153K,5,R,11.03.29.10.28.40,2
-- 103537280A,3,L,10.03.22.12.56.33,3
-- 103537280A,3,R,10.03.22.12.58.31,5
-- 104452232X,4,L,10.09.14.13.22.03,1
-- 104452232X,4,R,10.09.14.13.22.40,1
-- 100926322X,3,L,10.03.22.11.20.01,4
-- 100926322X,3,R,10.03.22.11.20.37,2
-- 101486229M,5,L,11.03.29.12.00.33,3
-- 101486229M,5,R,11.03.29.12.01.09,2
-- 101595920J,1,L,09.03.31.13.22.02,2
-- 101595920J,1,R,09.03.31.13.21.45,2
-- 103237663

-- 102986511N,1,R,09.05.05.09.53.30,2
-- 101089887F,1,L,09.05.05.13.53.47,2
-- 101089887F,1,R,09.05.05.13.54.15,2
-- 102511474P,1,L,09.05.05.09.49.02,5
-- 102511474P,1,R,09.05.05.09.49.27,1
-- 103238057F,4,L,10.11.22.12.36.44,2
-- 103238057F,4,R,10.11.22.12.37.15,2
-- 102984308T,3,L,10.05.04.09.37.26,2
-- 102984308T,3,R,10.05.04.09.37.50,2
-- 102986511N,3,L,10.05.03.11.38.59,2
-- 102986511N,3,R,10.05.03.11.39.25,1
-- 101429886L,5,L,11.05.09.12.06.58,5
-- 101429886L,5,R,11.05.09.12.07.28,3
-- 102146350J,6,L,11.10.24.11.33.36,4
-- 102146350J,6,R,11.10.24.11.33.54,4
-- 100154342X,2,L,09.09.01.13.23.34,3
-- 100154342X,2,R,09.09.01.13.24.07,3
........ DONE FOR 1600 .....
-- 102969575A,2,L,09.08.31.11.44.53,3
-- 102969380A,5,L,11.05.09.12.40.53,5
-- 102969380A,5,R,11.05.09.12.41.15,5
-- 100286848T,4,L,10.11.22.11.52.27,2
-- 100286848T,4,R,10.11.22.11.52.52,3
-- 102970039M,2,L,09.09.01.15.10.14,2
-- 101834831N,1,L,09.05.05.11.29.28,3
-- 101834831N,1,R,09.05.05.11.29.54,2
-- 102176410J,5,L,11.

-- 100486601N,1,L,09.05.05.10.11.29,1
-- 100486601N,1,R,09.05.05.10.12.12,2
-- 102735390J,6,L,11.10.24.10.27.35,2
-- 102735390J,6,R,11.10.24.10.28.03,2
-- 102735390J,1,L,09.05.05.09.57.42,2
-- 102735390J,1,R,09.05.05.09.58.03,2
-- 100256997F,1,L,09.05.05.10.24.30,1
-- 100256997F,1,R,09.05.05.10.25.00,1
-- 101429886L,1,L,09.05.04.12.31.16,2
-- 101429886L,1,R,09.05.04.12.31.41,3
-- 1008,2,L,09.09.01.13.45.08,3
-- 1008,2,R,09.09.01.13.45.38,3
-- 101100257F,2,L,09.09.01.13.36.09,1
-- 101100257F,2,R,09.09.01.13.36.36,2
-- 1007,2,L,09.08.31.15.04.27,2
-- 1007,2,R,09.08.31.15.04.46,2
-- 101466163K,1,L,09.05.05.11.47.47,1
-- 101466163K,1,R,09.05.05.11.48.07,2
-- 101466163K,2,L,09.09.01.11.24.00,1
-- 101466163K,2,R,09.09.01.11.24.20,1
-- 103039277F,1,L,09.05.05.11.25.36,2
-- 103039277F,1,R,09.05.05.11.26.02,2
-- 1006,1,L,09.05.05.12.45.50,2
-- 1006,1,R,09.05.05.12.46.11,2
-- 100973532X,1,L,09.05.05.11.53.48,4
-- 100973532X,1,R,09.05.05.11.54.26,4
-- 101578566L,1,L,09.05.05.12.06.06,2
-- 1015785